In [1]:
import fire
import os
import sys
import time
import json
import re
import random
import logging

import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from vllm import LLM, SamplingParams

from inference_utils import get_user_prompt, post_processing_answer
from embeddings_space import *

In [2]:
def main(
    model_path,
    # model_name,
    # peft_model: str = None,
    quantization: bool = False,
    load_in: str = "4bit",
    max_new_tokens=100,  # The maximum numbers of tokens to generate
    test_file: str = "datasets/math_test.json",
    seed: int = 42,  # seed value for reproducibility
    do_sample: bool = True,  # Whether or not to use sampling ; use greedy decoding otherwise.
    min_length: int = None,  # The minimum length of the sequence to be generated, input prompt + min_new_tokens
    use_cache: bool = True,  # [optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
    top_p: float = 1.0,  # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
    temperature: float = 1.0,  # [optional] The value used to modulate the next token probabilities.
    top_k: int = 50,  # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
    repetition_penalty: float = 1.0,  # The parameter for repetition penalty. 1.0 means no penalty.
    length_penalty: int = 1,  # [optional] Exponential penalty to the length that is used with beam-based generation.
    enable_azure_content_safety: bool = False,  # Enable safety check with Azure content safety api
    enable_sensitive_topics: bool = False,  # Enable check for sensitive topics using AuditNLG APIs
    enable_salesforce_content_safety: bool = True,  # Enable safety check with Salesforce safety flan t5
    max_padding_length: int = None,  # the max padding length to be used with tokenizer padding the prompts.
    use_fast_kernels: bool = False,  # Enable using SDPA from PyTroch Accelerated Transformers, make use Flash Attention and Xformer memory-efficient kernels
    log_filename: str = "log.txt",
    output_filepath: str = "jupyter_submission.csv",
    time_output_filepath: str = "time_submission.csv",
    **kwargs,
):
    logging.basicConfig(
        filename=log_filename,
        filemode='a',
        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
        datefmt='%H:%M:%S',
        level=logging.DEBUG
    )

    log = logging.getLogger(__name__)
    with open(test_file) as f:
        data = json.load(f)["data"]

    # Set the seeds for reproducibility
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)

    model = LLM(model=model_path, seed=seed)
    sampling_params = SamplingParams(n=1, best_of=1, temperature=temperature, top_p=top_p, stop_token_ids=[2], max_tokens=max_new_tokens)

    if use_fast_kernels:
        """
        Setting 'use_fast_kernels' will enable
        using of Flash Attention or Xformer memory-efficient kernels
        based on the hardware being used. This would speed up inference when used for batched inputs.
        """
        try:
            from optimum.bettertransformer import BetterTransformer

            model = BetterTransformer.transform(model)
        except ImportError:
            log.error(
                "Module 'optimum' not found. Please install 'optimum' it before proceeding."
            )
            results = []
    tokenizer_embedings, model_embedings = get_model_and_tokenizer()

    db = process_data(read_data())
    db_texts = db["information"].values.tolist()[:300]
    db["raw_texts"] = (
        "### Question:"
        + db["question"]
        + "### Choices: "
        + db["choices"]
        + "### Explanation: "
        + db["explanation"]
    )
    db_raw_texts = db["raw_texts"].values.tolist()[:300]
    log.info("Embedding database")
    db_embeddings = embedding_text(
        tokenizer=tokenizer_embedings, model=model_embedings, input_texts=db_texts
    )
    import gc

    gc.collect()

    # Dummy
    model.generate(get_user_prompt(data[0]), sampling_params)

    results = []
    times = []

    for idx, example in enumerate(data):
        log.info(f"Processing {idx}")
        start = time.time()
        relevant_examples = get_relevance_embeddings(
            db_embeddings,
            embedding_query_text(
                tokenizer=tokenizer_embedings,
                model=model_embedings,
                query_text=example["question"],
            ),
        )
        log.info("get relevant_question")
        relevant_examples = get_relevance_texts(
            input_texts=db_raw_texts, scores=relevant_examples, top_k=1
        )
        user_prompt = get_user_prompt(
            example, relevant_examples="\n".join(relevant_examples)
        )
        # user_prompt = get_user_prompt(example)
        id = example["id"]
        choices = example["choices"]

        output = model.generate(user_prompt, sampling_params)[0]

        prompt = output.prompt
        gen_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {gen_text!r}")

        answer_text = None

        for text in gen_text.split("###"):
            if 'Final choice' in text:
                answer_text = text
                break

        if answer_text is None:
            answer_text = gen_text

        log.info(f"Output text: {prompt + gen_text}")
        log.info(f"Gen text {gen_text}")
        log.info(f"Answer text {answer_text}")

        answer = post_processing_answer(answer_text, choices)
        e2e_inference_time = int((time.time() - start) * 1000)
        log.info(f'Inference time: {e2e_inference_time}')
        log.info(f"Answer {answer}")
        if answer is None:
            answer = random.choice(choices)
            log.info(f"Random Answer {answer}")
        results.append({"id": id, "answer": answer})
        times.append({"id": id, "time": e2e_inference_time})

    result_df = pd.DataFrame.from_dict(results)
    result_df.to_csv(output_filepath, index=False)

    time_df = pd.DataFrame.from_dict(times)
    time_df.to_csv(time_output_filepath, index=False)

In [ ]:
main(
    model_path='output',
    max_new_tokens=1024,
    temperature=0.1
)

INFO 11-30 15:26:34 llm_engine.py:72] Initializing an LLM engine with config: model='final', tokenizer='final', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=42)
INFO 11-30 15:26:45 llm_engine.py:207] # GPU blocks: 2016, # CPU blocks: 2048


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.48s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?\n### Choices: ['A. 4 500 000 đồng','B. 45 000 000 đồng','C. 50 000 000 đồng','D. 450 000 000 đồng']\n### Explanation: ", Generated text: 'Đầu tiên, chúng ta cần hiểu rằng 30% của một số là một phần của số đó. Để tìm số đó, chúng ta cần chia 15 000 000 đồng cho 30%.\n\nĐể thực hiện phép chia này, chúng ta cần chuyển 15 000 000 đồng thành phần trăm. Để làm điều này, chúng ta chia 15 000 000 đồng cho 100 và nhân kết quả với 100% để biểu thị kết quả dưới dạng phần trăm.\n\nKhi thực hiện phép chia này, chúng ta được kết quả là 50 000 000 đồng.\n\nVì vậy, nếu bán hết hàng thì cửa hàng thu được 50 000 000 đồng.\n### Final choice: C. 50 000 000 đồng\n'
full answer process C. 50 000 000 đồng


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.20s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Một người đi xe đạp từ A lúc 7 giờ với vận tốc 12km/h. Đến 8 giờ một người đi xe máy cũng từ A đuổi theo người đi xe đạp với vận tốc 42km/h. Hỏi người đi xe máy đuổi kịp người đi xe đạp lúc mấy giờ?\n### Choices: ['A. 24 phút','B. 1 giờ','C. 7 giờ 24 phút','D. 8 giờ 24 phút']\n### Explanation: ", Generated text: 'Để giải quyết câu hỏi này, chúng ta cần hiểu rằng người đi xe đạp sẽ đi được 12km trong một giờ và người đi xe máy sẽ đi được 42km trong một giờ. Điều này có nghĩa là sau mỗi giờ, người đi xe máy sẽ đi được 30km hơn người đi xe đạp.\n\nĐể tìm thời gian mà người đi xe máy đuổi kịp người đi xe đạp, chúng ta cần chia số km mà người đi xe máy đi được trong một giờ cho sự khác biệt về vận tốc giữa hai người. \n\nNgười đi xe máy đi được 42km trong một giờ và người đi xe đạp đi được 12km trong một giờ. Điều này

...

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.73s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Một xe lửa vượt qua cái cầu dài 450m hết 45 giây, vượt qua một cột điện hết 15 giây. Tính chiều dài của xe lửa.\n### Choices: ['A. 3m','B. 200m','C. 200m','D. 225m']\n### Explanation: ", Generated text: 'Đầu tiên, chúng ta cần hiểu rằng thời gian đi của xe lửa được chia thành hai phần: thời gian đi qua cầu và thời gian đi qua cột điện.\n\nThời gian đi qua cầu là 45 giây và thời gian đi qua cột điện là 15 giây, vì vậy tổng thời gian đi của xe lửa là 45 giây + 15 giây = 60 giây.\n\nChúng ta biết rằng thời gian đi của xe lửa được chia thành hai phần, vì vậy nếu chúng ta chia tổng thời gian đi của xe lửa cho chiều dài của cầu, chúng ta sẽ có thể tìm được chiều dài của xe lửa.\n\nChúng ta chia 60 giây cho 450m, vì vậy chiều dài của xe lửa sẽ là 60 giây / 450m = 0,13333...m.\n\nĐể chuyển đổi đơn vị đo lường này thành m

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.67s/it]


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\n### Question: Số lớn nhất trong các số: 5,216; 5,126; 5,621; 5,612 là:\n### Choices: ['A. 5,612','B. 5,621','C. 5,216','D. 5,126']\n### Explanation: ", Generated text: 'Để giải quyết câu hỏi này, chúng ta cần so sánh các số đã cho với nhau. \n\nBắt đầu bằng cách xem xét số 5,216. Đây là một số thập phân có ba chữ số sau dấu phẩy. \n\nTiếp theo, xem xét số 5,126. Đây cũng là một số thập phân có ba chữ số sau dấu phẩy. \n\nTiếp theo, xem xét số 5,621. Đây cũng là một số thập phân có ba chữ số sau dấu phẩy. \n\nCuối cùng, xem xét số 5,612. Đây cũng là một số thập phân có ba chữ số sau dấu phẩy. \n\nVì vậy, chúng ta cần xem xét các số này với nhau. \n\nBắt đầu bằng cách xem xét số 5,621 và 5,612. Hai số này có cùng phần nguyên là 5, nhưng số 6 ở hàng đơn vị khác nhau. Số 6 ở hàng đơn vị lớn hơn 1 so với số 6 ở hàng đơn vị của số 5

Processed prompts:   0%|                                                                                                                                                                     | 0/1 [00:00<?, ?it/s]